# Launch a new instance of ASReview

Make sure to add your own authenication token from [ngrok.com](https://dashboard.ngrok.com/auth) to the cell below.

ASReview can be opened using the link found in the output of the cell near the
end of the notebook.

In [ ]:
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "" #@param {type:"string"}

## Install ASReview and dependencies

In [ ]:
# installing packages... Colab has a lot of packages pre-installed and might
# throw an exception if you try to install new packages. This can be ignored.

!pip install pyngrok --quiet
!pip install --pre --upgrade asreview --quiet

# remove sample folder
!rm -r sample_data

In [ ]:
# imports
from pyngrok import ngrok
import os
import asreview
import ipywidgets as widgets
from IPython.display import display, Javascript

## Mount a google drive
To store the data and results of ASReview on your google drive, you will need to
mount your drive. The cells below mount your drive to the folder
`/content/drive/MyDrive/ASReview/`.

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')
! mkdir /content/drive/MyDrive/ASReview-colab

In [ ]:
# change ASReview project path to user files
os.environ['ASREVIEW_PATH'] = "/content/drive/MyDrive/ASReview-colab"

In [ ]:
# if you do not want to use the google drive, you can use the following line
# os.environ['ASREVIEW_PATH'] = "/content/asreview"

In [ ]:
# review the path
asreview.asreview_path()

## Run ngrok and ASReview
The cell below will start ngrok and ASReview. The last cell will continue to run
for as long as ASReview is active. To launch ASReview, click on the ngrok link
formatted like this: `http://xx-xx-xx-xx-xx.ngrok.io` which can be found in the
output of the second to last cell.

In [ ]:
# set ngrok authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Terminate open tunnels if exist
ngrok.kill()

In [ ]:
# Connect to the tunnel
session = ngrok.connect(port="80")

In [ ]:
# Get the public URL
print("Public URL:", session.public_url)

In [ ]:
# Create a button to open the link
out = widgets.Output()

def window_open_button(url):
    with out:
        display(Javascript(f'window.open("{url.tooltip}");'))

ss = widgets.Button(description="Launch ASReview", tooltip=session.public_url)
ss.on_click(window_open_button)

with out:
    display(ss)
out

In [ ]:
!asreview lab --port 80